## Unificació dels datasets amb l'informació estàtica de cada estat

Unificarem quatre conjunts de dades ja preprocessats que contenen informació a nivell estatal sobre llits hospitalaris, assegurança de salut, població i grups d'edats.

In [165]:
#pip install openpyxl

In [166]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

Carreguem els conjunts de dades

In [167]:
df_health = pd.read_csv('../data/preprocessed/acs_2018_health_insurance_coverage_estimates_cleaned.csv')
df_health.head()

,state,no_coverage,private_coverage,public_coverage,labor_cov_diff
0,Alabama,10.0,67.3,36.5,5.1
1,Alaska,14.4,65.5,31.0,5.6
2,Arizona,10.9,62.8,38.3,2.4
3,Arkansas,9.0,61.5,42.5,1.4
4,California,8.5,63.4,37.2,2.2


In [168]:
df_beds = pd.read_csv('../data/preprocessed/kff_usa_hospital_beds_per_capita_2018_cleaned.csv')
df_beds.head()

,state,state_local_government,non_profit,for_profit,total
0,Alaska,0.3,1.5,0.4,2.2
1,Alabama,1.4,0.8,0.9,3.1
2,Arkansas,0.3,2.2,0.7,3.2
3,Arizona,0.1,1.3,0.5,1.9
4,California,0.3,1.2,0.3,1.8


In [169]:
df_population = pd.read_csv('../data/preprocessed/population_cleaned.csv')
df_population.head()

,state,population,pop_density
0,AL,4887871,96.509389
1,AK,737438,1.291523
2,AZ,7171646,63.135855
3,AR,3013825,57.919684
4,CA,39557045,253.906502


In [170]:
df_ageGroups = pd.read_csv('../data/preprocessed/age_groups_cleaned.csv')
df_ageGroups.head()

,state,population,agegroup,pct_pop
0,AL,293203,0,0.059986
1,AL,297900,1,0.060947
2,AL,310495,2,0.063524
3,AL,315680,3,0.064584
4,AL,325220,4,0.066536


In [171]:
df_exogenous = pd.read_csv("..\data\preprocessed\static_stateMatrix.csv")
df_exogenous.head()

,state,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro,Census2019,Census2019_65Plus
0,Alabama,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
1,Alaska,0.79,83.41,7.29,8.51,67.41,32.59,731545.0,90588.0
2,Arizona,0.00,0.00,66.96,33.04,95.15,4.85,7278717.0,1307241.0
3,Arkansas,0.83,23.66,36.43,39.08,62.72,37.28,3017804.0,524237.0
4,California,1.87,29.21,17.24,51.69,98.11,1.89,39418470.0,5834998.0


### <span style="font-family:Courier New; color:#336633">**Adjust population by age group data**</span>

In [172]:
us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia', 'PR': 'Puerto Rico'
    }

df_population['state'] = df_population['state'].map(us_state_abbrev)
df_ageGroups['state'] = df_ageGroups['state'].map(us_state_abbrev)

In [173]:
# Pivot by agegroup, spreading pct_pop into columns
df_ageGroups = df_ageGroups.pivot(index='state', columns='agegroup', values='pct_pop').reset_index()
df_ageGroups.head()

agegroup,state,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Alabama,0.059986,0.060947,0.063524,0.064584,0.066536,0.069563,0.062098,0.062194,0.058984,0.064029,0.064341,0.068908,0.065134,0.055402,0.044380,0.030864,0.020142,0.018385
1,Alaska,0.072026,0.070590,0.067476,0.062735,0.071870,0.083630,0.077556,0.070070,0.058280,0.059067,0.060424,0.066780,0.061505,0.047567,0.031836,0.018840,0.010455,0.009293
2,Arizona,0.060786,0.062988,0.066431,0.066065,0.068628,0.072226,0.065238,0.063986,0.058779,0.060260,0.058372,0.061643,0.059148,0.054343,0.046912,0.033395,0.020960,0.019839
3,Arkansas,0.063157,0.064132,0.066622,0.065600,0.066904,0.068517,0.063073,0.063598,0.058678,0.061109,0.061216,0.065751,0.061816,0.053938,0.044279,0.031599,0.020428,0.019582
4,California,0.061716,0.062919,0.064413,0.064347,0.068554,0.079020,0.073766,0.069646,0.063217,0.064921,0.063298,0.063675,0.057194,0.046846,0.036209,0.024691,0.016768,0.018798


In [174]:
data_columns = [i for i in range(18)]

# We'll merge every two consecutive columns
merged_cols = []
for i in range(0, len(data_columns), 2):
    col1 = data_columns[i]
    col2 = data_columns[i + 1]
    merged = df_ageGroups[[col1, col2]].sum(axis=1)
    merged_cols.append(merged)

# Create new column names for the merged ranges
age_ranges = [
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39", "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]

# Build new DataFrame
df_ageGroups = pd.concat([df_ageGroups[['state']]] + merged_cols, axis=1)
df_ageGroups.columns = ['state'] + age_ranges

In [175]:
df_ageGroups.head()

,state,pop_0-9,pop_10-19,pop_20-29,pop_30-39,pop_40-49,pop_50-59,pop_60-69,pop_70-79,pop_80+
0,Alabama,0.120933,0.128108,0.136099,0.124291,0.123013,0.133249,0.120537,0.075243,0.038527
1,Alaska,0.142617,0.130210,0.155501,0.147626,0.117347,0.127204,0.109072,0.050675,0.019748
2,Arizona,0.123774,0.132496,0.140854,0.129224,0.119039,0.120016,0.113492,0.080306,0.040799
3,Arkansas,0.127289,0.132222,0.135421,0.126671,0.119787,0.126968,0.115755,0.075878,0.040010
4,California,0.124635,0.128760,0.147575,0.143412,0.128138,0.126974,0.104040,0.060900,0.035566


Realitzem un petit procés per evitar valors que falten a l'hora d'ajuntar els conjunts de dades.


Eliminem els valors relacionats amb l'estat de Puerto Rico, ja que aquests no són presents a tots els conjunts.

In [176]:
df_health = df_health[df_health['state'] != 'Puerto Rico']
df_population = df_population[df_population['state'] != 'Puerto Rico']
df_ageGroups = df_ageGroups[df_ageGroups['state'] != 'Puerto Rico']
df_exogenous = df_exogenous[df_exogenous['state'] != 'Puerto Rico']

Visualitzem les columnes dels diferentes datasets

In [177]:
print("Columnas del df_estado_1:", df_health.columns)
print("Columnas del df_estado_2:", df_beds.columns)
print("Columnas del df_estado_3:", df_population.columns)
print("Columnas del df_estado_4:", df_ageGroups.columns)
print("Columnas del df_estado_5:", df_exogenous.columns)

Columnas del df_estado_1: Index(['state', 'no_coverage', 'private_coverage', 'public_coverage',
       'labor_cov_diff'],
      dtype='object')
Columnas del df_estado_2: Index(['state', 'state_local_government', 'non_profit', 'for_profit', 'total'], dtype='object')
Columnas del df_estado_3: Index(['state', 'population', 'pop_density'], dtype='object')
Columnas del df_estado_4: Index(['state', 'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+'],
      dtype='object')
Columnas del df_estado_5: Index(['state', 'Low_SVI_CTGY', 'Moderate_Low_SVI_CTGY',
       'Moderate_High_SVI_CTGY', 'High_SVI_CTGY', 'Metro', 'Non-metro',
       'Census2019', 'Census2019_65Plus'],
      dtype='object')


Reanomenem i eliminem certes columnes per evitar duplicats i facilitar la comprensió de cada variable

In [178]:
df_beds = df_beds.rename(columns={'state_local_government': 'bedsState_local_government', 'non_profit': 'bedsNon_profit', 'for_profit': 'bedsFor_profit', 'total': 'bedsTotal' })
df_population = df_population.rename(columns={'population': 'population_state', 'pop_density': 'pop_density_state'})

df_exogenous = df_exogenous.drop(['Census2019', 'Census2019_65Plus'], axis = 1)

Unifiquem els 4 datasets basant-nos en la columna "state", que conté la informació de l'estat corresponent a cada fila.

In [179]:
df_health.shape[0] == df_beds.shape[0] == df_population.shape[0] == df_ageGroups.shape[0] == df_exogenous.shape[0]

True

In [180]:
merged_df = df_health.merge(df_beds, on='state', how='inner') \
    .merge(df_population, on='state', how='inner') \
    .merge(df_ageGroups, on='state', how='inner') \
    .merge(df_exogenous, on='state', how='inner')

print(merged_df.head())

        state  no_coverage  private_coverage  public_coverage  labor_cov_diff  \
0     Alabama         10.0              67.3             36.5             5.1   
1      Alaska         14.4              65.5             31.0             5.6   
2     Arizona         10.9              62.8             38.3             2.4   
3    Arkansas          9.0              61.5             42.5             1.4   
4  California          8.5              63.4             37.2             2.2   

   bedsState_local_government  bedsNon_profit  bedsFor_profit  bedsTotal  \
0                         1.4             0.8             0.9        3.1   
1                         0.3             1.5             0.4        2.2   
2                         0.1             1.3             0.5        1.9   
3                         0.3             2.2             0.7        3.2   
4                         0.3             1.2             0.3        1.8   

   population_state  ...  pop_50-59  pop_60-69  pop_70-7

Verifiquem que no hi hagi valors faltants després de la fusió

In [181]:
print(merged_df.isnull().sum())

state                         0
no_coverage                   0
private_coverage              0
public_coverage               0
labor_cov_diff                0
bedsState_local_government    0
bedsNon_profit                0
bedsFor_profit                0
bedsTotal                     0
population_state              0
pop_density_state             0
pop_0-9                       0
pop_10-19                     0
pop_20-29                     0
pop_30-39                     0
pop_40-49                     0
pop_50-59                     0
pop_60-69                     0
pop_70-79                     0
pop_80+                       0
Low_SVI_CTGY                  0
Moderate_Low_SVI_CTGY         0
Moderate_High_SVI_CTGY        0
High_SVI_CTGY                 0
Metro                         0
Non-metro                     0
dtype: int64


In [182]:
merged_df.head()

,state,no_coverage,private_coverage,public_coverage,labor_cov_diff,bedsState_local_government,bedsNon_profit,bedsFor_profit,bedsTotal,population_state,...,pop_50-59,pop_60-69,pop_70-79,pop_80+,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro
0,Alabama,10.0,67.3,36.5,5.1,1.4,0.8,0.9,3.1,4887871,...,0.133249,0.120537,0.075243,0.038527,8.99,15.97,43.99,31.04,76.84,23.16
1,Alaska,14.4,65.5,31.0,5.6,0.3,1.5,0.4,2.2,737438,...,0.127204,0.109072,0.050675,0.019748,0.79,83.41,7.29,8.51,67.41,32.59
2,Arizona,10.9,62.8,38.3,2.4,0.1,1.3,0.5,1.9,7171646,...,0.120016,0.113492,0.080306,0.040799,0.00,0.00,66.96,33.04,95.15,4.85
3,Arkansas,9.0,61.5,42.5,1.4,0.3,2.2,0.7,3.2,3013825,...,0.126968,0.115755,0.075878,0.040010,0.83,23.66,36.43,39.08,62.72,37.28
4,California,8.5,63.4,37.2,2.2,0.3,1.2,0.3,1.8,39557045,...,0.126974,0.104040,0.060900,0.035566,1.87,29.21,17.24,51.69,98.11,1.89


Desem la matriu de dades resultant de la combinació dels cinc conjunts de dades amb informació a nivell estatal

In [183]:
merged_df.to_csv('../data/preprocessed/dataMatrix/static_stateMatrix.csv', index=False)